# <div align="center">Heat transfert

<div align="center">
<B>Hassen Ghalila</B> (email: hassen.ghalila@devinci.fr), Pôle Leonard de Vinci<BR>    
Date: <B>2023</B>
</div>

### <div align="center">Tutorial 3
<div align="center">Exercice 4</div>

<B><I>Statment exo4</I></B><br>
Exhaust gases from a wire processing oven are discharged into a tall stack, and the gas and stack surface temperatures at the outlet of the stack must be estimated. Knowledge of the outlet gas temperature $T_{m,o}$ is useful for predicting the dispersion of effluents in the thermal plume, while knowledge of the outlet stack surface temperature $T_{s,o}$ indicates whether condensation of the gas products will occur. The thin-walled, cylindrical stack is 0.5 m in diameter and 6.0 m high. The exhaust gas flow rate is 0.5 $kg.s^{−1}$ , and the inlet temperature is 600°C.

<B><I>Factory</I></B><br>

<img src="Factory.png" align="center" width=300>

Consider conditions for which the ambient air temperature and wind velocity are 4°C and $5 m.s^{−1}$, respectively. Approximating the thermophysical properties of the gas as those of atmospheric air, estimate the outlet gas and stack surface temperatures for the given conditions.

<B><I>Input_Coeff.txt</I></B><br>
<img src="Data_Exo4.png" align="left" width=500>

In [12]:
import numpy as np
def Affect():
    global Visc,Pran_f,Pran_s,Body_f,Cond_f,N_Body_f
    global Vis_fi,Cp_fi,Cond_fi,Pran_fi,Body_fi,N_Body_fi
    global xmin,Diam_1,Diam_2,C1,m1,C2,m2,Length
    global u,u_inf,T_fl,T_s,T_fin,T_mo,T_mi
    global npt,Correl,Geom,Gas_FR
        
    Re = 5.E+5
    f = open('Data_Exo4.txt', 'r') # lire le fichier

    line=f.readline()
    while line[0]=="#":
        line=f.readline()    
    
    npt   = int(line.split(':')[0])
    Sym  = f.readline().split(':')[0]
    xmin  = float(f.readline().split(':')[0])*1E+0
    Diam_1  = float(f.readline().split(':')[0])*1E+0
    Length  = float(f.readline().split(':')[0])*1E+0
    Diam_2  = 30.E-6
    s = f.readline().split()
    C1,m1 = float(s[0]),float(s[1])
    s = f.readline().split()
    C2,m2 = float(s[0]),float(s[1])
    Geom = int(f.readline().split(':')[0])
    
    f.readline()
    T_fl   = float(f.readline().split(':')[0])
    T_s    = float(f.readline().split(':')[0])
    T_mi   = float(f.readline().split(':')[0])
    T_mo   = float(f.readline().split(':')[0])
    Pres   = float(f.readline().split(':')[0])
    u_inf  = float(f.readline().split(':')[0])*1E+0
    u_min  = float(f.readline().split(':')[0])*1E+0
    u_max  = float(f.readline().split(':')[0])*1E+0
    Gas_FR = float(f.readline().split(':')[0])*1E+0
    Correl = int(f.readline().split(':')[0])
    
    u = np.linspace(u_min,u_max,npt)

    line=f.readline()    
    Visc   = []
    Cond_f = []
    Pran_f = []
    Body_f = []
    Cp_fi   = []
    Vis_fi  = []
    Cond_fi = []
    Pran_fi = []
    Body_fi = []
    s = f.readline().split()
    Visc.append(float(s[0])*1.E-6)
    Cond_f.append(float(s[1]))
    Pran_f.append(float(s[2]))
    Body_f.append(s[4])
    line=f.readline()
    N_Body_f =1
    while True:
        s = line.split()
        if s[0][0]=='#': break
        Visc.append(float(s[0])*1.E-6)
        Cond_f.append(float(s[1]))
        Pran_f.append(float(s[2]))
        Body_f.append(s[4])
        N_Body_f += 1
        line=f.readline()
        if not line: break
    line=f.readline()
    N_Body_fi =0
    while True:
        s = line.split()
        if s[0][0]=='#': break
        Cp_fi.append(float(s[0]))
        Vis_fi.append(float(s[1])*1.E-7)
        Cond_fi.append(float(s[2]))
        Pran_fi.append(float(s[3]))
        Body_fi.append(s[5])
        N_Body_fi += 1
        line=f.readline()
        if not line: break
    f.close() 

In [13]:
Affect()

## 1) Calculating $T_{m,o}$ from Eq. (2.57) 

$
\large
\begin{equation}
\frac{T_{∞} - T_{m,o}}{T_{∞} - T_{m,i}} = exp^{- \frac{1}{\dot{m}c_{p}R_{tot}}} 
\hspace{2cm} with \hspace{2cm}
R_{tot} = \frac{1}{h_{i}} + \frac{1}{h_{o}}
\label{Rate_Eq} \tag{1}
\end{equation}
$

### Internal flow :

$
\begin{equation}
Reynolds~number~(Eq~(2.38)):\hspace{1cm} Re_{D} = \frac{4\dot{m}}{\pi D\mu} = 33827
\end{equation}
$

=> The flow of the internal fluide is trubulent

We can use the **Dittus-Boelter** for cooling Eq. (2.62) 

$
\begin{equation}
Nu_{D} = 0,023Re_{D}^{4/5}Pr^{0,3} =87,273
\label{Nusselt_nu_Int} \tag{2}
\end{equation}
$

<BR><div align="center"> => $h_{i} = 10,2 W/m^{2}/K$ </div>

In [14]:
def Dittus_Boelter():
    global Re_Di,h_bari,Nu_Di
    Re_Di  = np.zeros((N_Body_fi))
    Nu_Di  = np.zeros((N_Body_fi))
    h_bari = np.zeros((N_Body_fi))
    for i in range(N_Body_f):
        Re_Di[i] = 4*Gas_FR/np.pi/Diam_1/Vis_fi[i]
        Nu_Di[i] = 0.023*Re_Di[i]**(4./5.)*Pran_fi[i]**0.3
        h_bari[i] = Cond_fi[i]*Nu_Di[i]/Diam_1

In [16]:
Dittus_Boelter()

### External flow

$
\begin{equation}
Reynolds~number~(Eq~(2.34)):\hspace{1cm} Re_{D} = \frac{VD}{\nu} = 94660
\end{equation}
$

=> The flow of the external fluide is trubulent

We can use the **Churchill Bernstein** for cooling Eq. (2.36) 

$
\begin{equation}
Nu_{D} = \left(0,60 + \frac{0,387Ra_{D}^{1/6}}{\left[1+\left(0,559/4,16\right)^{9/16}\right]^{8/27}}\right)^{2}     = 205,22 
\label{Nusselt_nu_Ext} 
 \tag{3}
\end{equation}
$

<BR><div align="center"> => $h_{o} = 13,9 W/m^{2}/K$ </div>

In [17]:
def Churchill_Bernstein():
    global Re_D,Nu_D,h_bar,q_dot
    
    Re_D  = np.zeros((N_Body_f))
    Nu_D  = np.zeros((N_Body_f))
    h_bar = np.zeros((N_Body_f))
#    q_dot = np.zeros((N_Body))
    for i in range(N_Body_f):
        Re_D[i] = u_inf*Diam_1/Visc[i]
        Nu_D[i] = 0.3 + (0.62*np.sqrt(Re_D[i])*Pran_f[i]**(1/3))/(1+(0.4/Pran_f[i])**(2/3))**0.25*(1+(Re_D[i]/282000)**(5/8))**(4/5)
        h_bar[i]  = Cond_f[i]*Nu_D[i]/Diam_1
#        q_dot[i] = h_bar[i]*np.pi*Diam_1*(T_s - T_fl)

In [18]:
Churchill_Bernstein()

## 2) Deducing $T_{s,o}$ under the assumption of steady state

$
h_{i}(T_{m,o} - T_{s,o}) = h_{o}(T_{s,o} - T_{∞})
$

<br>=> $T_{s,o} = 232 °C$


In [19]:
Rtot = (1/h_bari[0] + 1/h_bar[0])/Length/(np.pi*Diam_1)
T_mo = T_fl - (T_fl - T_mi)*np.exp(-1./(Gas_FR*Cp_fi[0]*Rtot))
T_so = (h_bari[0]*T_mo + h_bar[0]*T_fl)/(h_bari[0]+h_bar[0])

In [20]:

def Sorties():
    f = open("MyOutPut.txt","w")
    print('\n-----------------------------------------------')
    print("Results wrote also in the file : ",f.name)
    print('-------------------------------------------------')

    
    print('\n----- Results for internal fluide ----------')     
    f.write('\n----- Results for internal fluide ---------- \n')     
    Output_txt='{0: <10}'.format('Material')+'{0: <13}'.format('  Re_D') \
              +'{0: <10}'.format('Nu_u')+'{0: <14}'.format('h_bar(W/m2/K)')
    print(Output_txt)     
    f.write(Output_txt+'\n')
    MyString=3*'{:6.4e}  '
    for i in range(N_Body_f):  
        print(f'{Body_fi[i]:10}', 3*'%6.4e  ' %(Re_Di[i],Nu_Di[i],h_bari[i]))
        f.write('{:6} : '.format(Body_fi[i]) + MyString.format(Re_Di[i],Nu_Di[i],h_bari[i])+'\n')

    print('\n----- Results for external fluide ----------')     
    f.write('\n----- Results for external fluide ---------- \n')     
    Output_txt='{0: <10}'.format('Material')+'{0: <13}'.format('  Re_D') \
              +'{0: <10}'.format('Nu_u')+'{0: <14}'.format('h_bar(W/m2/K)')
    print(Output_txt)     
    f.write(Output_txt+'\n')
    MyString=3*'{:6.4e}  '
    for i in range(N_Body_f):  
        print(f'{Body_f[i]:10}', 3*'%6.4e  ' %(Re_D[i],Nu_D[i],h_bar[i]))
        f.write('{:6} : '.format(Body_f[i]) + MyString.format(Re_D[i],Nu_D[i],h_bar[i])+'\n')

    MyString=2*'{:6.4e}  '
    Output_txt='  {0: <9}'.format('T_mo(°C)')+'  {0: <11}'.format('T_so(°C)')
    print('\n'+Output_txt)     
    f.write('\n'+Output_txt+'\n')
    print(2*'%6.4e  ' %(T_mo-273.15,T_so-273.15))
    f.write(MyString.format(T_mo-273.15,T_so-273.15)+'\n')

    f.close()

In [21]:
Sorties()


-----------------------------------------------
Results wrote also in the file :  MyOutPut.txt
-------------------------------------------------

----- Results for internal fluide ----------
Material    Re_D       Nu_u      h_bar(W/m2/K) 
Air_Tb_mi  3.3827e+04  8.7273e+01  1.0124e+01  

----- Results for external fluide ----------
Material    Re_D       Nu_u      h_bar(W/m2/K) 
Air_T_m    9.4661e+04  2.0522e+02  1.3873e+01  

  T_mo(°C)   T_so(°C)   
5.4332e+02  2.3153e+02  
